In [199]:
import pandas as pd
import nltk
import numpy as np

In [200]:
file_name = 'data/Travel-Dataset-5000--master/5000TravelQuestionsDataset.xlsx'
col_names = ['questions', 'a', 'b']
dataset = pd.read_excel(file_name, header=None, names=col_names)
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   questions  5000 non-null   object
 1   a          5000 non-null   object
 2   b          5000 non-null   object
dtypes: object(3)
memory usage: 117.3+ KB
None


In [161]:
dataset['a'].unique()

array(['TTD', 'TGU', 'ACM', 'TRS', 'WTH', 'FOD', 'ENT', 'TGU\n', 'TTD\n',
       '\nENT'], dtype=object)

In [162]:
dataset['b'].unique()

array(['TTDSIG', 'TTDOTH', 'TGUHEA', 'ACMOTH', 'TRSTRN', 'TGUBAN',
       'WTHTMP', 'TGUNEI', 'TTDSPO', 'ACMAPA', 'TRSOTH', 'TRSAIR',
       'TRSROU', 'TGUOTH', 'ACMRES', 'ACMBUN', 'TRSCRS', 'WTHDRC',
       'TRSRNT', 'TGUTOP', 'FODOTH', 'TGUVIS', 'FODAUT', 'TTDTRI',
       'TGUPLN', 'TRSTAX', 'WTHOTH', 'TRSBUS', 'ACMHOT', 'TGULUG',
       'FODBAK', 'TTDSPA', 'FODBRE', 'TGUATT', 'ENTCLB', 'TGUAVE',
       'TGUTEL', 'TGUCIG', 'TTDSHP', 'TRSTCD', 'ACMBEA', 'FODCOT',
       'TGUAPT', 'TRSLIC', 'TGULAU', 'TTDGYM', 'FODCAT', 'FODBAR',
       'TGUHOL', 'ENTFES', 'TGURUL', 'TGURES', 'WTHTMP\n', 'ACMCAR',
       'FODFMA', 'ENTSHW', 'TRSGAS', '\nTGULAU', 'TRSOTH\n', 'WTHSNW',
       'ENTMUS', 'ENTSPO', 'FODBAK\n', 'TRSAIR\n', 'TGUWEB', 'TRSDRV',
       'FODFCA', 'TGUCIG\n', 'ENTOTH', 'TTDOTH\n', 'WTHOTH\n', 'TTDSIG\n',
       'TGUOTH\n', 'TTDSHP\n', 'TRSROU\n', 'TTDSPO\n', '\nACMOTH',
       'ACMOTH\n', '\nWTHOTH'], dtype=object)

# PreProcessing

In [165]:
dataset.head()

,questions,a,b
0,What are the special things we (husband and me...,TTD,TTDSIG
1,What are the companies which organize shark fe...,TTD,TTDOTH
2,Is it safe for female traveller to go alone to...,TGU,TGUHEA
3,What are the best places around Cape Town for ...,TTD,TTDSIG
4,What are the best places to stay for a family ...,ACM,ACMOTH


In [166]:
### Lowercase

In [167]:
test_id = 2
dataset['questions'].dropna(inplace=True)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   questions  5000 non-null   object
 1   a          5000 non-null   object
 2   b          5000 non-null   object
dtypes: object(3)
memory usage: 117.3+ KB


In [168]:
test_id = 9
dataset['questions'][test_id]

'What are the best beaches for shelling in CapeTown?'

In [204]:
import re

def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """
    
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)    
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text

dataset['clean_questions'] = [lowercase(question) for question in dataset['questions']]
dataset['clean_questions'][test_id]

'what are the best beaches for shelling in capetown?'

In [170]:
# def lowercase(x):
#     return x.lower()

# dataset['questions'] = [clean_text(question) for question in dataset['questions']]
# dataset['lc_questions'][test_id]

In [171]:
# import string

# def remove_punctuation(x):
#     return "".join([char for char in x if char not in string.punctuation])

# dataset['questions'] = [remove_punctuation(question) for question in dataset['questions']]
# dataset['questions'][test_id]

In [172]:
# from nltk.corpus import stopwords
# stop_words = stopwords.words('english')
# from nltk import word_tokenize

# def remove_stopwords(x):
#     words = word_tokenize(x)
#     return " ".join([word for word in words if word not in stop_words])

# dataset['questions'] = [remove_stopwords(question) for question in dataset['questions']]
# dataset['questions'][test_id]
# print(stop_words)

In [173]:
# from nltk.stem.porter import PorterStemmer
# porter = PorterStemmer()
# def stemming(x):
#     filtered_words = word_tokenize(x['questions'])
#     stemmed = [porter.stem(word) for word in filtered_words]
#     return " ".join(stemmed)

# dataset['questions'] = [remove_stopwords(question) for question in dataset['questions']]
# dataset['questions'][test_id]

# Create features

## Lemmatize and create BOW

In [205]:
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer()
def lemmatize(x):
    filtered_words = nltk.word_tokenize(x)
    lemmatized = [lemmatizer.lemmatize(word) for word in filtered_words]
    return " ".join(lemmatized)

dataset['lem_questions'] = [lemmatize(question) for question in dataset['clean_questions']]
dataset['lem_questions'][test_id]

'what are the best beach for shelling in capetown ?'

## POS Tags

In [212]:
from nltk import pos_tag
def pos_tagging(x):
    words = nltk.word_tokenize(x)
    lst = [ r[1] for r in pos_tag(words)] 
    return ' '.join(lst)

dataset['pos_questions'] = [pos_tagging(question) for question in dataset['questions']]
dataset['pos_questions'][test_id]

'WP VBP DT JJS NNS IN VBG IN NNP .'

## Naming Entities

In [213]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree

def get_continuous_chunks(text):
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    prev = None
    continuous_chunk = []
    current_chunk = []

    for i in chunked:
        if type(i) == Tree:
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue

    if continuous_chunk:
        named_entity = " ".join(current_chunk)
        if named_entity not in continuous_chunk:
            continuous_chunk.append(named_entity)
    
    def remove_null(x):
        if '' in x:
            x.remove('')
        return x

    lst = remove_null(continuous_chunk)
    return ' '.join(lst)

txt = "Barack Obama is a great person." 
txt2 = "Who is Dulan?"
print (get_continuous_chunks(txt2))



dataset['ne_questions'] = [get_continuous_chunks(question) for question in dataset['questions']]
dataset['ne_questions'][test_id]

Dulan


'CapeTown'

In [256]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

def get_count_vect(documents):
    vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
    X = vectorizer.fit_transform(documents).toarray()
    print(X.shape)
    return X

print(get_count_vect(dataset['questions']))

(5000, 1090)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# SVM - Normal train

In [234]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X, y = dataset['lem_questions'],dataset['a']

tfidf = TfidfVectorizer(max_features=5000)
tfidf.fit(X)

le = LabelEncoder()
le.fit(y)


cv = KFold(n_splits=10, random_state=1, shuffle=True)
fold = 0
accuracies = []
for train_index, test_index in cv.split(X):
    fold += 1
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    y_train_e = le.transform(y_train)
    y_test_e = le.transform(y_test)
    
    X_train_tfidf = tfidf.transform(X_train)
    X_test_tfidf = tfidf.transform(X_test)
    print(X_train_tfidf.shape)
    
    SVM = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(X_train_tfidf,y_train_e)
    predictions_SVM = SVM.predict(X_test_tfidf)
    acc = accuracy_score(predictions_SVM, y_test_e)*100
    accuracies.append(acc)
    print("Fold - {} - {} - {:.2f}".format(fold, "SVM Accuracy Score -> ",acc))
    
print("Mean {:.2f} Std {:.2f}".format(np.mean(accuracies), np.std(accuracies)))

(4500, 5000)
Fold - 1 - SVM Accuracy Score ->  - 83.20
(4500, 5000)
Fold - 2 - SVM Accuracy Score ->  - 84.60
(4500, 5000)
Fold - 3 - SVM Accuracy Score ->  - 81.00
(4500, 5000)
Fold - 4 - SVM Accuracy Score ->  - 82.00
(4500, 5000)
Fold - 5 - SVM Accuracy Score ->  - 83.40
(4500, 5000)
Fold - 6 - SVM Accuracy Score ->  - 83.80
(4500, 5000)
Fold - 7 - SVM Accuracy Score ->  - 84.00
(4500, 5000)
Fold - 8 - SVM Accuracy Score ->  - 84.80
(4500, 5000)
Fold - 9 - SVM Accuracy Score ->  - 83.60
(4500, 5000)
Fold - 10 - SVM Accuracy Score ->  - 83.40
Mean 83.38 Std 1.08


In [235]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X, y = dataset['pos_questions'],dataset['a']

tfidf = TfidfVectorizer(max_features=5000)
tfidf.fit(X)

le = LabelEncoder()
le.fit(y)


cv = KFold(n_splits=10, random_state=1, shuffle=True)
fold = 0
accuracies = []
for train_index, test_index in cv.split(X):
    fold += 1
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    y_train_e = le.transform(y_train)
    y_test_e = le.transform(y_test)
    
    X_train_tfidf = tfidf.transform(X_train)
    X_test_tfidf = tfidf.transform(X_test)
    print(X_train_tfidf.shape)
    
    SVM = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(X_train_tfidf,y_train_e)
    predictions_SVM = SVM.predict(X_test_tfidf)
    acc = accuracy_score(predictions_SVM, y_test_e)*100
    accuracies.append(acc)
    print("Fold - {} - {} - {:.2f}".format(fold, "SVM Accuracy Score -> ",acc))
    
print("Mean {:.2f} Std {:.2f}".format(np.mean(accuracies), np.std(accuracies)))

(4500, 31)
Fold - 1 - SVM Accuracy Score ->  - 38.80
(4500, 31)
Fold - 2 - SVM Accuracy Score ->  - 40.00
(4500, 31)
Fold - 3 - SVM Accuracy Score ->  - 39.80
(4500, 31)
Fold - 4 - SVM Accuracy Score ->  - 42.20
(4500, 31)
Fold - 5 - SVM Accuracy Score ->  - 41.80
(4500, 31)
Fold - 6 - SVM Accuracy Score ->  - 40.00
(4500, 31)
Fold - 7 - SVM Accuracy Score ->  - 39.40
(4500, 31)
Fold - 8 - SVM Accuracy Score ->  - 37.00
(4500, 31)
Fold - 9 - SVM Accuracy Score ->  - 40.20
(4500, 31)
Fold - 10 - SVM Accuracy Score ->  - 33.80
Mean 39.30 Std 2.29


In [261]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from scipy.sparse import coo_matrix, csr_matrix, hstack

le = LabelEncoder()
y = le.fit_transform(dataset['a'])

X_lem = dataset['lem_questions']
tfidf_lem = TfidfVectorizer(max_features=5000)
tfidf_lem.fit(X_lem)

X_pos = dataset['pos_questions']
# tfidf = TfidfVectorizer(max_features=50)
# tfidf.fit(X_pos)

X_ne = dataset['ne_questions']
print(get_count_vect(X_pos))

print(type(tfidf_lem.transform(X_lem)))
XX = csr_matrix(hstack([tfidf_lem.transform(X_lem) ,get_count_vect(X_pos), get_count_vect(X_ne)]))
print(type(XX))
XX.shape

(5000, 26)
[[1 1 0 ... 0 1 0]
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]]
<class 'scipy.sparse.csr.csr_matrix'>
(5000, 26)
(5000, 322)
<class 'scipy.sparse.csr.csr_matrix'>


(5000, 5348)

In [280]:
dataset['lem_questions'].shape

(5000,)

In [262]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


cv = KFold(n_splits=10, random_state=1, shuffle=True)
fold = 0
accuracies = []
for train_index, test_index in cv.split(XX):
    fold += 1
    X_train, X_test = XX[train_index], XX[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    print(X_train.shape)

    
    SVM = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(X_train,y_train)
    predictions_SVM = SVM.predict(X_test)
    acc = accuracy_score(predictions_SVM, y_test)*100
    accuracies.append(acc)
    print("Fold - {} - {} - {:.2f}".format(fold, "SVM Accuracy Score -> ",acc))
    
print("Mean {:.2f} Std {:.2f}".format(np.mean(accuracies), np.std(accuracies)))

(4500, 5348)
Fold - 1 - SVM Accuracy Score ->  - 79.20
(4500, 5348)
Fold - 2 - SVM Accuracy Score ->  - 82.40
(4500, 5348)
Fold - 3 - SVM Accuracy Score ->  - 79.60
(4500, 5348)
Fold - 4 - SVM Accuracy Score ->  - 79.60
(4500, 5348)
Fold - 5 - SVM Accuracy Score ->  - 80.80
(4500, 5348)
Fold - 6 - SVM Accuracy Score ->  - 81.60
(4500, 5348)
Fold - 7 - SVM Accuracy Score ->  - 81.20
(4500, 5348)
Fold - 8 - SVM Accuracy Score ->  - 79.00
(4500, 5348)
Fold - 9 - SVM Accuracy Score ->  - 80.80
(4500, 5348)
Fold - 10 - SVM Accuracy Score ->  - 79.80
Mean 80.40 Std 1.07


# Word embeddings

## FastText

In [133]:
import fasttext
import os
model_name='fasttext_skipgram_travel_questions.bin'

if not os.path.exists(model_name):
    dataset['questions'].to_csv('questions.txt', sep='.', header=False, index=False)
    model = fasttext.train_unsupervised('questions.txt', model='skipgram')
    model.save_model(model_name)
    print("Model saved as {}".format(model_name))
else:
    print("\"{}\" - model loaded".format(model_name))
    model = fasttext.load_model(model_name)

"fasttext_skipgram_travel_questions.bin" - model loaded


In [134]:
print('hello' in model.words)   # list of words in dictionary
print(model['hello']) # get the vector of the word 'king'
print(len(model.words), len(model['hello']))

False
[ 7.14299676e-05  2.32432321e-01 -2.40925714e-01 -1.74242929e-01
 -4.39231321e-02  2.45131522e-01 -2.27550954e-01  3.51612344e-02
 -1.35040674e-02  6.65467754e-02  1.80292074e-02  4.08698320e-02
  1.47805631e-01 -4.09955978e-02  1.14978559e-01 -4.33138423e-02
 -1.80182517e-01 -1.58651307e-01 -2.19959438e-01 -3.89570266e-01
 -1.93506658e-01  3.53299528e-01  1.32871151e-01 -1.09432288e-01
 -7.85724819e-02 -3.23186889e-02  2.63973564e-01 -2.09144980e-01
  1.40900716e-01  4.18550760e-01  1.00273587e-01 -2.01780409e-01
  1.07219130e-01 -2.64418364e-01 -1.07324362e-01  1.89055175e-01
 -1.23978466e-01 -6.86219707e-02 -1.82332143e-01 -1.99690199e-04
  1.38345420e-01  3.61972660e-01 -8.87458958e-03 -5.23991622e-02
  2.63315067e-02 -2.84253269e-01  5.83582968e-02 -1.83719546e-01
 -7.05709029e-03  1.42239183e-01  3.36414427e-02  1.66355848e-01
 -1.53241102e-02  1.18326567e-01  2.25987196e-01  6.47004843e-02
 -4.14141901e-02 -3.55386846e-02 -5.03547899e-02  3.76824252e-02
  8.85521099e-02  1

## Gensim Doc2Vec - Using Sentence Vectors

In [309]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(dataset['questions'])]
model = Doc2Vec(documents, vector_size=1000, window=2, min_count=1, workers=4)

def doc2vec(x):
    return np.array(model.infer_vector(x.split(' ')))

In [310]:
doc2vec('hello world').shape

(1000,)

In [311]:
# _id = 3
# vector = model.infer_vector(dataset['questions'][_id].split(' '))
# print(vector)
# print(dataset['questions'][_id])

dataset['doc2vec_questions'] = np.array([doc2vec(question) for question in dataset['questions']])
dataset['doc2vec_questions'][test_id]
dataset['doc2vec_questions'].shape

(5000,)

In [312]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

XX = np.array([doc2vec(question) for question in dataset['questions']])
print(XX.shape)
cv = KFold(n_splits=10, random_state=1, shuffle=True)
fold = 0
accuracies = []
for train_index, test_index in cv.split(XX):
    fold += 1
    X_train, X_test = XX[train_index], XX[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    print(X_train.shape)

    
    SVM = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(X_train,y_train)
    predictions_SVM = SVM.predict(X_test)
    acc = accuracy_score(predictions_SVM, y_test)*100
    accuracies.append(acc)
    print("Fold - {} - {} - {:.2f}".format(fold, "SVM Accuracy Score -> ",acc))
    
print("Mean {:.2f} Std {:.2f}".format(np.mean(accuracies), np.std(accuracies)))

(5000, 1000)
(4500, 1000)
Fold - 1 - SVM Accuracy Score ->  - 21.20
(4500, 1000)
Fold - 2 - SVM Accuracy Score ->  - 27.80
(4500, 1000)
Fold - 3 - SVM Accuracy Score ->  - 25.20
(4500, 1000)
Fold - 4 - SVM Accuracy Score ->  - 23.40
(4500, 1000)
Fold - 5 - SVM Accuracy Score ->  - 22.80
(4500, 1000)
Fold - 6 - SVM Accuracy Score ->  - 24.40
(4500, 1000)
Fold - 7 - SVM Accuracy Score ->  - 22.00
(4500, 1000)
Fold - 8 - SVM Accuracy Score ->  - 24.00
(4500, 1000)
Fold - 9 - SVM Accuracy Score ->  - 26.20
(4500, 1000)
Fold - 10 - SVM Accuracy Score ->  - 22.60
Mean 23.96 Std 1.91


# Create Dataset

In [135]:
## Train test split

In [137]:
X_test_tfidf[0]

<1x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 16 stored elements in Compressed Sparse Row format>

In [138]:
accuracies

[83.39999999999999,
 83.8,
 80.2,
 82.39999999999999,
 82.8,
 83.39999999999999,
 82.8,
 84.6,
 83.6,
 82.39999999999999]

In [139]:
import numpy as np

np.std([1,2])

0.5

In [140]:
## Word2vec

In [141]:
import gensim
data = ['data']
model = gensim.models.Word2Vec(data, min_count=1, sg=1, window = 3)

# A NN classifier s.a. an LSTM for classification

In [323]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 20
# This is fixed.
EMBEDDING_DIM = 100

In [324]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(dataset['questions'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 5584 unique tokens.


In [330]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X = tokenizer.texts_to_sequences(dataset['questions'].values)
print(X[0])
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

y_nn = pd.get_dummies(dataset['a']).values
print('Shape of label tensor:', y_nn.shape)


[4, 7, 2, 321, 105, 31, 1838, 17, 68, 9, 20, 71, 6, 194, 48, 32, 22, 376, 111]
Shape of data tensor: (5000, 20)
Shape of label tensor: (5000, 10)


In [331]:
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from tensorflow.keras import Sequential

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64



In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

cv = KFold(n_splits=10, random_state=1, shuffle=True)
fold = 0
accuracies = []
for train_index, test_index in cv.split(XX):
    fold += 1
    X_train, X_test = XX[train_index], XX[test_index]
    y_train, y_test = y_nn[train_index], y_nn[test_index]
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/5
64/64 [==============================] - 109s 2s/step - loss: 1.8625 - accuracy: 0.2343 - val_loss: 1.8829 - val_accuracy: 0.0756
Epoch 2/5
64/64 [==============================] - 115s 2s/step - loss: 1.7757 - accuracy: 0.2551 - val_loss: 1.9834 - val_accuracy: 0.0756
Epoch 3/5
64/64 [==============================] - 124s 2s/step - loss: 1.7746 - accuracy: 0.2612 - val_loss: 2.0528 - val_accuracy: 0.0756
Epoch 4/5
64/64 [==============================] - 125s 2s/step - loss: 1.7757 - accuracy: 0.2627 - val_loss: 1.9533 - val_accuracy: 0.0756
Epoch 1/5
52/64 [=======================>......] - ETA: 23s - loss: 1.7689 - accuracy: 0.2524